In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [29]:
df = pd.read_csv("../data-translated/Checking_translated.csv", encoding='iso-8859-1')
df.head()

,Date,nps,Comment,comment
0,1/19/2023 22:35,10,Muy bien.,Very good.
1,1/19/2023 22:36,9,Es rÃ¡pido,It's fast
2,1/19/2023 22:36,9,"ES BUENO, PERO DE REPENTE SE BLOQUEA","IT'S GOOD, BUT SUDDENLY IT BLOCKS"
3,1/19/2023 22:40,10,rapidez,speed
4,1/19/2023 22:44,10,TODO MUY CLARO,ALL VERY CLEAR


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     5001 non-null   object
 1   nps      5001 non-null   object
 2   Comment  4995 non-null   object
 3   comment  4899 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [31]:
df.drop(columns="Date", inplace=True)
df.dropna(inplace=True)

In [32]:
bins = [6, 7, 8, 10]
labels = pd.cut(df.nps.astype(int), bins=bins, labels=["M", "R", "B"])
labels.value_counts()

B    4880
R       6
M       1
Name: nps, dtype: int64

In [33]:
sents = ['too easy', 'agile', 'rapid', 'FOR THE RAPIDITY', 'super facil', 'TOO EASY', 'and', 'fast and efficient page', 'Very efficient',
 'It is very easy and fast.',  'great',  'Everything is very simple and fast, thank you',  'Easy to use',  'convenient and fast',
 'by the ease',  'service on webb',  'BECAUSE OF THE EASE OF THE PROCESS',  'Friendly',  'EASILY DOING IT, I AVOID PROCEDURES AT THE AIRPORT',
 'Flexibility.',  'good service',  'IT IS WITH WHOM I FLY',  'all excellent',  'Easy access',  'Simplicity',  'THE SERVICE IS FAST',
 'VERY EASY',  'SPEED AND PRACTICALITY',  'for the speed', 'very good service', 'good and easy to use', 'Very easy',
  'The whole process was quick and easy.',  'FOR ITS SPEED', 'ALL VERY WELL', 'excellent service', 'The website is very practical',
 'very well', 'Of course', 'AGILE AND EASY PROCESS']

In [34]:
def custom_standardization(data):
    lower = tf.strings.lower(data)
    return lower

In [57]:
dataset = tf.data.Dataset.from_tensor_slices(df["comment"].to_numpy()).batch(32)
# list(dataset.as_numpy_iterator())

In [58]:
vocab_size = 10000
vectorizer = TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=vocab_size,
    # output_mode='int',
    output_sequence_length=100)

In [59]:
vectorizer.adapt(df.comment.to_numpy())

In [60]:
vectorizer.vocabulary_size()

1143

In [61]:
print(vectorizer.get_vocabulary()[5:15])

output = vectorizer([["i like it a lot"]])
print(output.numpy()[0, :6])

['fast', 'it', 'is', 'the', 'good', 'service', 'to', 'its', 'for', 'simple']
[ 19  51   6  37 121   0]


In [62]:
embedding_dim=16
n_classes = len(df.nps.unique())

embedding_layer = Embedding(vocab_size, embedding_dim, name="embedding")
# model = Sequential([
#   vectorizer,
#   E,
#   GlobalAveragePooling1D(),
#   Dense(16, activation='relu'),
#   Dense(n_classes)
# ])

In [63]:
from sklearn.model_selection import train_test_split

train_samples, val_samples, train_labels, val_labels = train_test_split(df.comment, pd.get_dummies(df.nps), test_size=0.2, random_state=42)

In [75]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# y_train = np.array([[s] for s in train_labels])
# y_val = np.array([[s] for s in val_labels])

In [78]:
y_train.shape

(3915, 8)

In [71]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.GlobalMaxPooling1D()(embedded_sequences)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(n_classes, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_max_pooling1d_4 (Glo  (None, 16)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_14 (Dense)            (None, 8)                 136       
                                                           

In [72]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer="rmsprop", metrics=["acc"]
)

In [73]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [74]:
model.fit(x_train,
    y_train,
    epochs=15,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback])

ValueError: Data cardinality is ambiguous:
  x sizes: 3915
  y sizes: 8
Make sure all arrays contain the same number of samples.